In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
from astropy import units as u

c = const.c.value
G = const.G.value
M_sun = const.M_sun.value
M_earth = const.M_earth.value

In [11]:
def rel_len(x_1, x_2):
    r_vec = x_1 - x_2
    r = np.linalg.norm(r_vec)
    return r

def fourdot(V_1, V_2):
    return V_1[0]*V_2[0] - V_1[1]*V_2[1] - V_1[2]*V_2[2]

def findR_S(m_1, m_2, DL):
    if DL == True:
        c = 1
        G = 1
    else:
        c = const.c.value
        G = const.G.value
    R_S = 2 * (m_1 + m_2) * G/c**2
    return R_S

def findrCM(x_1, x_2, m_1, m_2):
    M = m_1 + m_2
    r_cm = (m_1*x_1 + m_2*x_2)/M
    return r_cm

def γ(v, DL):
    if DL == True:
        c = 1
    else:
        c = const.c.value
    return 1/np.sqrt(1 - np.dot(v,v)/c**2)

def four_P(p, m, DL):
    if DL == True:
        c = 1
    else:
        c = const.c.value
    return np.array([np.sqrt(np.dot(p, p)*c**2 + m**2*c**4)/c, p[0], p[1]])

def rotate(V, θ):
    rot = np.array([[np.cos(θ), -np.sin(θ)], [np.sin(θ), np.cos(θ)]])
    return np.dot(rot, V)

In [12]:

def accelerationPM(x_1, x_2, P_1, P_2, m_1, m_2, DL, α = 1, order=1):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c.value
        G = const.G.value
    
    if α == 0:
        P_1[0] = m_1*c
        P_2[0] = m_2*c

    P = np.array([P_1, P_2])
    E_1 = P_1[0]*c
    E_2 = P_2[0]*c
    Es = np.array([E_1, E_2])
    c_1 = m_1**2*m_2**2 - 2*(fourdot(P_1, P_2))**2
    ξ = E_1*E_2/(E_1 + E_2)**2


    def E(i):
        return Es[i-1]

    def E_dot(i):
        return (p(i,1)*p_dot(i,1) + p(i,2)*p_dot(i,2))*c**2/E(i)

    def p(i, σ):
        return P[i-1,σ]

    def p_dot(i, σ):
        result = G*r_vec[σ]/r**3 * (m_1**2*m_2**2*c**4 - 2*fourdot(P_1, P_2)**2)/(E(1)*E(2))
        if i == 1:
            result = result
        elif i == 2:
            result = - result
        else:
            print('Error: i is different than 1 or 2')
        return result

    def P_dot(i):
        return np.array([E_dot(i)/c, p_dot(i, 1),  p_dot(i, 2)])

    def q_dot(i, σ):
        j = 3 - i
        return p(i, σ)*c**2/E(i) - α * G/r * (4*fourdot(P_1, P_2)/(E(1)*E(2)) * (E(j)/E(i)*p(i, σ) - p(j, σ)) + c**2*(m_1**2*m_2**2*c**4 - 2*fourdot(P_1,P_2)**2)/(E(1)*E(2))**2 * E(j)/E(i)*p(i, σ))
    
    

    def V2PM_p(i, σ):
        j = 3 - i
        return -G**2/(r*E_1*E_2)**2*( ( 3/4*(m_1**2*m_2**2 - 5*fourdot(P_1, P_2)**2)*(m_1 + m_2)/c**2 + c_1*(E_1 + E_2)/(E_1*E_2)*(c_1*(ξ-1)/(2*(E_1*E_2)) - 4*fourdot(P_1, P_2)/c**2) )*E(j)*p(i, σ)/E(i)*c**2 + E_1*E_2*( 30/4*fourdot(P_1, P_2)*(E(j)/E(i)*p(i, σ) - p(j, σ))*(m_1 + m_2)/c**2 + ( 4*fourdot(P_1, P_2)*(E(j)/E(i)*p(i, σ) - p(j, σ))*(E_1 + E_2)/(E_1*E_2) - c_1*p(i, σ)/(E_1*E_2*E(i))*c**2 + c_1*(E_1 + E_2)/(E_1*E_2)**2*p(i, σ)*c**2 )*(c_1*(ξ-1)/(2*(E_1*E_2)) - 4*fourdot(P_1, P_2)/c**2) + c_1*(E_1 + E_2)(E_1*E_2)*( (4*fourdot(P_1, P_2)*(E(j)/E(i)*p(i, σ) - p(j, σ))*(ξ - 1) - c_1*p(i, σ)/(E(i)*(E_1 + E_2)**3)*c**2*(E(j)*(E_1 + E_2) - 2*E_1*E_2) )/(2*E_1*E_2) + c_1*(ξ-1)/(E_1*E_2)**2*E(j)/E(i)*p(i, σ)*c**2 + 4*((E(j)/E(i)*p(i, σ)) - p(j, σ))/c**2 ) ) ) 
    
    def V2PM_q(i, σ):
        result = -2*G**2/r**3*r_vec[σ]/(E_1*E_2)*( 3/4*(m_1**2*m_2**2 - 5*fourdot(P_1, P_2)**2)*(m_1 + m_2)/c**2 + c_1*(E_1 + E_2)/(E_1*E_2)*(c_1*(ξ-1)/(2*(E_1*E_2)) - 4*fourdot(P_1, P_2)/c**2) )
        if i == 1:
            result = result
        elif i == 2:
            result = - result
        else:
            print('Error: i is different than 1 or 2')
        return result
    
    def V2PM_p_t(i, σ):
        return -(2*)

    def a(i, σ, order):
        j = 3 - i
        a1 = p_dot(i, σ)*c**2/E(i) + α * ( - p(i, σ)*E_dot(i)*c**2/E(i)**2 + G/r**2*r_dot*(4*fourdot(P_1, P_2)/(E(1)*E(2)) * (E(j)/E(i)*p(i, σ) - p(j, σ)) + c**2*(m_1**2*m_2**2*c**4 - 2*fourdot(P_1,P_2)**2)/(E(1)*E(2))**2 * E(j)/E(i)*p(i, σ)) - G/r * (4*((fourdot(P_dot(1), P_2) + fourdot(P_1, P_dot(2)))/(E(1)*E(2)) - fourdot(P_1, P_2)/(E(1)*E(2))**2*(E_dot(1)*E(2) + E(1)*E_dot(2)))*(E(j)/E(i)*p(i, σ) - p(j, σ)) + 4 * fourdot(P_1, P_2)/(E(1)*E(2)) * ((E_dot(j)/E(i) - E(j)*E_dot(i)/E(i)**2)*p(i, σ) + E(j)/E(i)*p_dot(i, σ) - p_dot(j, σ)) - (4* fourdot(P_1, P_2)*(fourdot(P_dot(1), P_2) + fourdot(P_1, P_dot(2)))/(E(1)*E(2))**2 - 2*(m_1**2*m_2**2*c**4 - 2*fourdot(P_1, P_2)**2)/(E(1)*E(2))**3*(E_dot(1)*E(2) + E(1)*E_dot(2)))*E(j)/E(i)*p(i, σ)*c**2 + (m_1**2*m_2**2*c**4 - 2*fourdot(P_1, P_2)**2)/(E(1)*E(2))**2*( (E_dot(j)/E(i) - E(j)*E_dot(i)/E(i)**2)*p(i, σ) + E(j)/E(i)*p_dot(i, σ))*c**2) )
        a2 = a1 + 
        if order == 1:
            a = a1
        elif order == 2:
            a = a2
        return any

    r = rel_len(x_1, x_2)
    r_vec = np.array([0, x_1[0] - x_2[0], x_1[1] - x_2[1]])

    v_1 = np.array([q_dot(1,1), q_dot(1,2)])
    v_2 = np.array([q_dot(2,1), q_dot(2,2)])

    r_dot = (r_vec[1]*(v_1[0] - v_2[0]) + r_vec[2]*(v_1[1] - v_2[1]))/r

    a_1 = np.array([a(1, 1), a(1, 2)])
    a_2 = np.array([a(2, 1), a(2, 2)])
    return np.array([v_1, v_2, a_1, a_2])


def boost(v_1, v_2, a_1, a_2, dt):
    v_1 += a_1*dt
    v_2 += a_2*dt
    return np.array([v_1, v_2])

def move(x_1, x_2, v_1, v_2, dt):
    x_1 += v_1*dt
    x_2 += v_2*dt
    return np.array([x_1, x_2])

def punch(v_1, v_2, m_1, m_2, DL):
    p_1 = γ(v_1, DL)*v_1*m_1
    p_2 = γ(v_2, DL)*v_2*m_2
    P_1 = four_P(p_1, m_1, DL)
    P_2 = four_P(p_2, m_2, DL)
    return np.array([P_1, P_2])


In [ ]:
def symplectic_integrate_step(x_1, x_2, v_1, v_2, m_1, m_2, dt, coeffs, DL, α):
    for ai,bi in coeffs.T:
        P_1, P_2 = punch(v_1, v_2, m_1, m_2, DL)
        v_1, v_2, a_1, a_2 = accelerationPM(x_1, x_2, P_1, P_2, m_1, m_2, DL=DL, α=α)
        v_1 += bi * a_1 * dt
        v_2 += bi * a_2 * dt
        x_1 += ai * v_1 * dt
        x_2 += ai * v_2 * dt
    return x_1, x_2, v_1, v_2

def symplectic_integrate(s, t_max, dt, coeffs, DL=False, α=1):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c.value
        G = const.G.value
    
    pos1 = np.zeros((2, int(t_max/dt)))
    pos2 = np.zeros((2, int(t_max/dt)))
    pos_CM = np.zeros((2, int(t_max/dt)))

    vel1 = np.zeros((2, int(t_max/dt)))
    vel2 = np.zeros((2, int(t_max/dt)))

    x_1, x_2, v_1, v_2, m_1, m_2 = s
    a_1, a_2 = 0, 0

    R_S = findR_S(m_1, m_2, DL)
    r_init = rel_len(x_1, x_2)

    for i in range(int(t_max/dt)):
        pos1[:,i] = x_1
        pos2[:,i] = x_2
        pos_CM[:,i] = findrCM(x_1, x_2, m_1, m_2)

        vel1[:,i] = v_1
        vel2[:,i] = v_2

        x_1, x_2, v_1, v_2 = symplectic_integrate_step(x_1, x_2, v_1, v_2, m_1, m_2, dt, coeffs, DL, α)

        if rel_len(x_1, x_2) <= 5*R_S:
            print('Code interrupted: Radial Dip')
            pos1, pos2, pos_CM = pos1[:,0:i], pos2[:,0:i], pos_CM[:,0:i]
            break
        elif rel_len(x_1, x_2) >= 5*r_init:
            print('Code interrupted: Max distance')
            pos1, pos2, pos_CM = pos1[:,0:i], pos2[:,0:i], pos_CM[:,0:i]
            break
    return pos1, pos2, pos_CM

δ = np.math.pow(2.0, 1.0/3.0)
ruth4 = np.array([[0.5, 0.5*(1.0-δ), 0.5*(1.0-δ), 0.5],
                  [0.0,         1.0,          -δ, 1.0]]) / (2.0 - δ)
ruth3 = np.array([[2.0/3.0, -2.0/3.0, 1.0], [7.0/24.0, 0.75, -1.0/24.0]])
leap2 = np.array([[0.5, 0.5], [0.0, 1.0]])

In [ ]:
# import matplotlib.pyplot as plt

# def symplectic_integrate_step2(qvt0, accel, dt, coeffs):
#     q,v,t = qvt0
#     for ai,bi in coeffs.T:
#         v += bi * accel(q,v,t) * dt
#         q += ai * v * dt
#         t += ai * dt
#     return q,v,t

# def symplectic_integrate2(qvt0, accel, t, coeffs):
#     q = np.empty_like(t)
#     v = np.empty_like(t)
#     qvt = qvt0
#     q[0] = qvt[0]
#     v[0] = qvt[1]
#     for i in xrange(1, len(t)):
#         qvt = symplectic_integrate_step(qvt, accel, t[i]-t[i-1], coeffs)
#         q[i] = qvt[0]
#         v[i] = qvt[1]
#     return q,v

# # c = np.math.pow(2.0, 1.0/3.0)
# # ruth4 = np.array([[0.5, 0.5*(1.0-c), 0.5*(1.0-c), 0.5],
# #                   [0.0,         1.0,          -c, 1.0]]) / (2.0 - c)
# # ruth3 = np.array([[2.0/3.0, -2.0/3.0, 1.0], [7.0/24.0, 0.75, -1.0/24.0]])
# # leap2 = np.array([[0.5, 0.5], [0.0, 1.0]])

# # accel = lambda q,v,t: -q
# # qvt0 = (1.0, 0.0, 0.0)
# # tmax = 2.0 * np.math.pi
# # N = 36

# fig, ax = plt.subplots(1, figsize=(6, 6))
# ax.axis([-1.3, 1.3, -1.3, 1.3])
# ax.set_aspect('equal')
# ax.set_title(r"Phase plot $(y(t),y'(t))$")
# ax.grid(True)
# t = np.linspace(0.0, tmax, 3*N+1)
# q,v = symplectic_integrate(qvt0, accel, t, leap2)
# ax.plot(q, v, label='leap2 (%d steps)' % (3*N), color='black')
# t = np.linspace(0.0, tmax, N+1)
# q,v = symplectic_integrate(qvt0, accel, t, ruth3)
# ax.plot(q, v, label='ruth3 (%d steps)' % N, color='red')
# q,v = symplectic_integrate(qvt0, accel, t, ruth4)
# ax.plot(q, v, label='ruth4 (%d steps)' % N, color='blue')
# ax.legend(loc='center')
# fig.show()

In [ ]:
def findMaxPos(pos):
    xmax, xmin, ymax, ymin = max(pos[0]), min(pos[0]), max(pos[1]), min(pos[1])
    return xmax, xmin, ymax, ymin

def plotLims(pos):
    start_pos = np.array([pos[0][0], pos[1][0]])
    end_pos = np.array([pos[0][-1], pos[1][-1]])

    xmax, xmin, ymax, ymin = findMaxPos(pos)
    max_pos = max(max(abs(start_pos)),max(abs(end_pos)))

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def plotLimsTwoBody(pos1, pos2):
    start_pos1 = np.array([pos1[0][0], pos1[1][0]])
    end_pos1 = np.array([pos1[0][-1], pos1[1][-1]])
    start_pos2 = np.array([pos2[0][0], pos2[1][0]])
    end_pos2 = np.array([pos2[0][-1], pos2[1][-1]])

    x1max, x1min, y1max, y1min = findMaxPos(pos1)
    x2max, x2min, y2max, y2min = findMaxPos(pos2)

    xmax = max(x1max, x2max)
    xmin = min(x1min, x2min)
    ymax = max(y1max, y2max)
    ymin = min(y1min, y2min)

    max_pos1 = max(max(abs(start_pos1)),max(abs(end_pos1)))
    max_pos2 = max(max(abs(start_pos2)),max(abs(end_pos2)))
    max_pos = max(max_pos1, max_pos2)

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def orbPlotter(pos1, pos2, posCM, parameters = np.zeros(8), aspect = 1, filename='', CM = True, save = False, show = True, DL = False, figsize=(8,8)):
    x_1 = pos1
    x_2 = pos2 
    x_cm = posCM
    if CM == True:
        x_1 = x_1 - x_cm
        x_2 = x_2 - x_cm
        x_cm = np.zeros_like(x_1)

    fig, ax = plt.subplots(figsize=figsize)
    xlim, ylim = plotLimsTwoBody(x_1, x_2)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # ax.plot(x_cm[0], x_cm[1], 'g:', label = 'CM', lw=0.4)
    ax.plot(x_cm[0], x_cm[1], 'g:', lw=0.5)
    ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
    ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    # ax.plot(x_1[0], x_1[1],'b', label = 'm_1', lw=0.75)
    ax.plot(x_1[0], x_1[1],'b', lw=1)
    ax.plot(x_1[0][0], x_1[1][0], 'b.', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'bo', label = 'm_1 stop',markersize=15)

    # ax.plot(x_2[0], x_2[1],'r', label = 'm_2', lw=0.75)
    ax.plot(x_2[0], x_2[1],'r', lw=1)
    ax.plot(x_2[0][0], x_2[1][0], 'r.', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'ro', label = 'm_2 stop',markersize=15)

    if DL == True:
        ax.set_xlabel('$R_S$', fontsize = 15)
        ax.set_ylabel('$R_S$', fontsize = 15)
    else:
        ax.set_xlabel('$x \ [\mathrm{m}]$', fontsize = 15)
        ax.set_ylabel('$y \ [\mathrm{m}]$', fontsize = 15)  
    ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    ax.set_title(f'{filename}', fontsize = 20)
    ax.legend(facecolor='grey', fontsize = 12, bbox_to_anchor=(1.01,1), loc='upper left')

    # d = {
    #      'E:':      parameters[0],
    #      '|p_1|:':  parameters[1],
    #      '|p_2|:':  parameters[2],
    #      '|L_1|:':  parameters[3],
    #      '|L_2|:':  parameters[4],
    #      'R_S:':    parameters[5],
    #      'b:':      parameters[6],
    #      'θ:':      parameters[7],
    #     }

    # text = nice_string_output(d, extra_spacing=3, decimals=2)
    # add_text_to_ax(1.04, 0, text, ax, fontsize=12)

    plt.title(f'{filename}', fontsize=25, y=1.08)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/{filename}.png', dpi=300, transparent = False)
    return

def orbPlotter_lims(pos1, pos2, posCM, xlim, ylim, parameters = np.zeros(8), aspect = 1, filename='', CM = False, save = False, show = True, DL = False, figsize=(8,8)):
    x_1 = pos1
    x_2 = pos2 
    x_cm = posCM
    if CM == True:
        x_1 = x_1 - x_cm
        x_2 = x_2 - x_cm
        x_cm = np.zeros_like(x_1)

    fig, ax = plt.subplots(figsize=figsize)
    # xlim, ylim = plotLimsTwoBody(x_1, x_2)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    ax.plot(x_cm[0], x_cm[1], 'g:',label = 'CM')
    ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
    ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    ax.plot(x_1[0], x_1[1],'b', label = 'm_1')
    ax.plot(x_1[0][0], x_1[1][0], 'b.', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'bo', label = 'm_1 stop',markersize=15)

    ax.plot(x_2[0], x_2[1],'r', label = 'm_2')
    ax.plot(x_2[0][0], x_2[1][0], 'r.', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'ro', label = 'm_2 stop',markersize=15)

    if DL == True:
        ax.set_xlabel('$R_S$', fontsize = 15)
        ax.set_ylabel('$R_S$', fontsize = 15)
    else:
        ax.set_xlabel('$x \ [\mathrm{m}]$', fontsize = 15)
        ax.set_ylabel('$y \ [\mathrm{m}]$', fontsize = 15)  
    ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    ax.set_title(f'{filename}', fontsize = 20)
    ax.legend(facecolor='grey', fontsize = 12, bbox_to_anchor=(1.01,1), loc='upper left')

    # d = {
    #      'E:':      parameters[0],
    #      '|p_1|:':  parameters[1],
    #      '|p_2|:':  parameters[2],
    #      '|L_1|:':  parameters[3],
    #      '|L_2|:':  parameters[4],
    #      'R_S:':    parameters[5],
    #      'b:':      parameters[6],
    #      'θ:':      parameters[7],
    #     }

    # text = nice_string_output(d, extra_spacing=3, decimals=2)
    # add_text_to_ax(1.04, 0, text, ax, fontsize=12)

    plt.title(f'{filename}', fontsize=25, y=1.08)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/{filename}.png', dpi=300, transparent = False)
    return